# Device Actions Fixtures

Generate test fixtures from the device actions datasets

In [33]:
import pathlib
from synthetic_home import synthetic_home
import shutil
import yaml

DATASET_DIR = pathlib.Path("../datasets/")
DEVICES_DIR = DATASET_DIR / "devices-v3"
DEVICE_ACTIONS_DIR = DATASET_DIR / "device-actions-v2"
DEVICE_ACTIONS_FIXTURES_DIR = DATASET_DIR / "device-actions-v2-fixtures"

shutil.rmtree(DEVICE_ACTIONS_FIXTURES_DIR, ignore_errors=True)
DEVICE_ACTIONS_FIXTURES_DIR.mkdir(exist_ok=True)

homes_count = 0
devices_count = 0
sentences_count = 0
device_type_sentences = {}

for devices_file in DEVICES_DIR.glob("*.yaml"):
   home_id = devices_file.name.split(".")[0]
   home = synthetic_home.load_synthetic_home(devices_file)

   home_dir = DEVICE_ACTIONS_FIXTURES_DIR / home_id
   home_dir.mkdir(exist_ok=True)

   inventory = synthetic_home.build_inventory(home)

   fixtures = home_dir / "_fixtures.yaml"
   fixtures.write_text(inventory.to_yaml())

   homes_count += 1
   category_tasks = {}
   for actions_file in (DEVICE_ACTIONS_DIR / home_id).glob("*.yaml"):
      devices_count += 1
      device_actions = yaml.load(actions_file.read_text(), Loader=yaml.CSafeLoader)
      device = device_actions["device"]
      category = device["device_type"]
      if category not in category_tasks:
         category_tasks[category] = []

      actions = device_actions["actions"]
      if "actions" in actions:
         actions = actions["actions"]
      for action_data in actions:
         sentences = action_data["sentences"]
         category_tasks[category].append(sentences)
      sentences_count += len(sentences)
      device_type_sentences[category] = device_type_sentences.get(category, 0) + len(sentences)

   for category, tasks in category_tasks.items():
      data = {
         "category": category,
         "tests": [
         {
            "sentences": sentences
         }
         for sentences in tasks
         ]
      }
      category_file = home_dir / f"{category}.yaml"
      category_file.write_text(yaml.dump(data, sort_keys=False, explicit_start=True))

print(homes_count, devices_count, sentences_count)
print(yaml.dump(device_type_sentences, explicit_start=True, sort_keys=False))


40 479 1908
---
light: 619
smart-plug: 109
light-dimmable: 636
hvac: 103
exhaust-fan: 33
smart-speaker: 141
switch: 25
smart-sprinkler: 56
water-valve: 28
garage-door: 18
smart-tv: 95
heat-pump: 10
smart-lock: 15
smart-blinds: 3
vacuum: 11
fan-oscilating: 6

